# 3.DATA PROCESSING AND ANALYTICS

## Importing the libraries needed

In [1]:
import pandas as pd
import numpy as np
import shapely
import helper2
import calendar
import os, sys
from sklearn.cluster import OPTICS


## Importin MovinPy ,ST-Visions and Evolving Clusters from Data Stories Unipi.

In [2]:
sys.path.append(os.path.join('DataStories-UniPi', 'ST-Visions'))
import st_visualizer
import express as viz_express
import geom_helper as viz_helper
import bokeh.palettes as bokeh_palettes

In [3]:
sys.path.append(os.path.join('DataStories-UniPi', 'EvolvingClusters'))
import EvolvingClusters
from EvolvingClusters import evolving_clusters

In [4]:
sys.path.append(os.path.join('DataStories-UniPi', 'MovinPy'))
import helper


## 3.1 Trajectory Segmentation Using Helper. 

In [192]:
ais_feb=pd.read_csv('./data/unipi_ais_feb_2018_clean.csv',header='infer',sep=';') #read data from csv files
ais_mar=pd.read_csv('./data/unipi_ais_mar_2018_clean.csv',header='infer',sep=';')
df=ais_feb.append(ais_mar) #combine the two datasets (ais February-March)
df=helper.getGeoDataFrame_v2(df, crs='epsg:4326') # convert the df to a geodf
df=df.rename(columns={'timestmp':'ts'}) # rename column in order to match required input.
df['datetime'] = pd.to_datetime(df.ts,unit='ms') #adding the datetime feature computed by the unix date.
df['traj_id']=0 #adding the traj_id column in order to match required input

tr=helper2.create_trajectories(df) #create trajectories using helper
tr=tr.rename(columns={'ts':'timestamp'})# rename column in order to match required input
fix_trajectories=helper2.fix_trajectories(tr) #fixing trajectories using helper
#The actual trajectory segmentation.
trajectories_seg=helper2.temporal_segmentation(fix_trajectories, TEMPORAL_THRESHOLD = 60*60*12, CARDINALITY_THRESHOLD = 3)

(Initial) Number of segments: 1
(Final-Useful) Number of port-based segments produced: 1
(Initial) Number of port-based segments: 1
(Intermediate) Number of temporal-gap-based segments: 6
(Final-Useful) Number of trips produced: 5


In [6]:
trajectories_seg.loc[:, 'timestamp'] = pd.to_datetime(trajectories_seg.timestamp,unit='ms')#converting timestamp to datetime
trajectories_seg=trajectories_seg.rename(columns={'timestamp':'ts'}) #rename column

In [193]:
trajectories_seg

,index,mmsi,timestamp,geom,lon,lat,velocity,ais_type,ais_status,ais_heading,ais_turn,ais_course,acceleration,datetime,traj_id,label,trip_id
0,0,237017200,1517436000000,POINT (23.56900 37.95528),23.568995,37.955283,0.0,NaN,NaN,NaN,NaN,0.1,0.0,2018-01-31 22:00:00,0,0,0
1,1,237008100,1517436002000,POINT (23.64102 37.94490),23.641023,37.944897,0.0,NaN,NaN,NaN,NaN,NaN,0.0,2018-01-31 22:00:02,0,0,0
2,2,249136000,1517436004000,POINT (23.59589 37.95244),23.595890,37.952443,0.0,NaN,NaN,348.0,NaN,313.1,0.0,2018-01-31 22:00:04,0,0,0
3,3,239737000,1517436004000,POINT (23.62549 37.94088),23.625490,37.940883,0.0,NaN,NaN,152.0,NaN,0.0,0.0,2018-01-31 22:00:04,0,0,0
4,4,239299000,1517436005000,POINT (23.65033 37.93317),23.650333,37.933167,0.0,NaN,NaN,187.0,NaN,140.0,0.0,2018-01-31 22:00:05,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10194336,10194339,305192000,1522443598000,POINT (23.55832 37.89019),23.558317,37.890185,0.1,NaN,NaN,91.0,NaN,191.0,NaN,2018-03-30 20:59:58,0,0,4
10194337,10194337,237008100,1522443598000,POINT (23.64021 37.94150),23.640205,37.941497,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-30 20:59:58,0,0,4
10194338,10194340,239737000,1522443598000,POINT (23.62734 37.90562),23.627343,37.905618,25.4,NaN,NaN,339.0,NaN,338.2,NaN,2018-03-30 20:59:58,0,0,4
10194339,10194341,239634000,1522443599000,POINT (23.58251 37.95108),23.582505,37.951080,1.1,NaN,NaN,NaN,NaN,235.7,NaN,2018-03-30 20:59:59,0,0,4


# 3.3 Temporal alignment and Resampling Using Helper

In [8]:
#TEMPORAL ALIGNMENT AND RESAMPLING
resampled_and_aligned=helper2.temporal_alignment_v2(trajectories_seg) 

In [9]:
resampled_and_aligned=resampled_and_aligned.rename(columns={'datetime':'ts'})

In [10]:
len(resampled_and_aligned) #the length of the resampled dataset.

83459

# 3.4 Implement the OPTICS clustering algorithm in order to find hot-spots and display them using ST-Visions.

## Implement Optics with haversine distance metric. 

In [5]:
ra_df=pd.read_csv('./data/samples_resampled_aligned.csv',header='infer',sep=';')

In [6]:
ra_geodf=helper.getGeoDataFrame_v2(ra_df, crs='epsg:4326')
ra_geodf=ra_geodf.drop(columns=['index','datetime'])

In [7]:
ra_geodf.head()

,mmsi,timestamp,geom,lon,lat,velocity,ais_type,ais_status,ais_heading,ais_turn,ais_course,acceleration,traj_id,label,trip_id
0,239924000,1517436240000,POINT (23.64048 37.94680),23.640483,37.946800,0.0,NaN,262.0,NaN,NaN,282.0,0.0,0,0,0
1,218031000,1517436300000,POINT (23.62627 37.79145),23.626268,37.791448,14.0,NaN,153.0,NaN,NaN,150.9,0.0,0,0,0
2,218031000,1517436420000,POINT (23.63116 37.78454),23.631157,37.784543,14.5,NaN,153.0,NaN,NaN,151.4,0.0,0,0,0
3,237017200,1517436480000,POINT (23.56906 37.95525),23.569063,37.955247,0.0,NaN,NaN,NaN,NaN,0.1,0.0,0,0,0
4,241024000,1517436540000,POINT (23.65010 37.93167),23.650102,37.931672,0.0,NaN,332.0,NaN,NaN,311.3,0.0,0,0,0


In [8]:
len(ra_geodf)

73068

In [14]:
gdf_radians_resampled_and_aligned = np.radians(ra_df [['lat', 'lon']]) #creating the radians
optics_resampled_and_aligned  = OPTICS(max_eps=1/6371, min_samples=len(ra_df )//50, metric='haversine').fit(gdf_radians_resampled_and_aligned )#implementing the optics clustering algorithm
set(optics_resampled_and_aligned .labels_)
helper2.get_clusters_centers(gdf_radians_resampled_and_aligned , optics_resampled_and_aligned .labels_)

array([[37.94691256, 23.64038312],
       [37.94731144, 23.63740902],
       [37.9427454 , 23.64046713],
       [37.94420766, 23.63208578],
       [37.95457064, 23.55113343],
       [37.94993253, 23.54616344],
       [37.9316622 , 23.65010351],
       [37.93275919, 23.6812189 ]])

In [18]:
set(optics_resampled_and_aligned .labels_)

{-1, 0, 1, 2, 3, 4, 5, 6, 7}

## Display the clusters found as Hot-Spots in a Map using ST-Visions. 

In [16]:
tmp = ra_df.copy()
tmp.loc[:, 'cluster_labels'] = optics_resampled_and_aligned.labels_
tmp.cluster_labels = tmp.cluster_labels.apply(str)
points = st_visualizer.st_visualizer()
points.set_data(tmp.copy())
points.create_canvas('Prototype Plot')
cmap = points.add_categorical_colormap('Reds', 'cluster_labels')
points.add_glyph(color=cmap, legend_group='cluster_labels')
points.add_map_tile(provider='CARTODBPOSITRON')
points.add_hover_tooltips([('timestamp', '@ts')])

points.show_figures(notebook=True, notebook_url='http://localhost:8888')

Loading BokehJS ...

## Display only the clusters centers using ST-Visions 

In [17]:
tmp=pd.DataFrame(helper2.get_clusters_centers(gdf_radians_resampled_and_aligned , optics_resampled_and_aligned .labels_))
tmp=tmp.rename(columns={0:'lat',1:'lon'})
a=pd.DataFrame(optics_resampled_and_aligned .labels_)
a=a.rename(columns={0:'labels'})
a=a.labels.unique()
tmp['cluster_labels']=a[0:8]
tmp.cluster_labels = tmp.cluster_labels.apply(str)
points = st_visualizer.st_visualizer()
points.set_data(tmp.copy())
points.create_canvas('Prototype Plot')
cmap = points.add_categorical_colormap('Reds', 'cluster_labels')
points.add_glyph(color=cmap, legend_group='cluster_labels')
points.add_map_tile(provider='CARTODBPOSITRON')
points.add_hover_tooltips([('label', '@cluster_labels')])
points.show_figures(notebook=True, notebook_url='http://localhost:8888')

Loading BokehJS ...

# 3.5 Finding Evolving Clusters using the Evolving Clusters Algorithm and Display the results using ST-Visions.

In [10]:
final=ra_geodf
final['timestamp']=final['timestamp'].floordiv(1000) #convert unix timestamp from ms to sec.
final=final.rename(columns={'timestamp':'ts'})
final.sort_values('ts', inplace=True)

## Implementing Evolving Clusters Algorithm

In [12]:
[res_mcs, res_mc] = evolving_clusters(final, distance_threshold=1000, min_cardinality=1, time_threshold=1, disable_progress_bar=False)

100%|██████████| 71910/71910 [06:03<00:00, 197.94it/s]


In [13]:
for row in res_mcs.itertuples(): # display the clusters found
    print(row)

Pandas(Index=0, clusters=(241024000,), st=Timestamp('2018-02-01 01:07:00'), et=Timestamp('2018-02-01 01:08:00'))
Pandas(Index=0, clusters=(241024000,), st=Timestamp('2018-02-06 01:23:00'), et=Timestamp('2018-02-06 01:24:00'))
Pandas(Index=0, clusters=(241087000,), st=Timestamp('2018-02-14 18:37:00'), et=Timestamp('2018-02-14 18:38:00'))
Pandas(Index=0, clusters=(241024000,), st=Timestamp('2018-02-17 00:24:00'), et=Timestamp('2018-02-17 00:25:00'))
Pandas(Index=0, clusters=(241024000,), st=Timestamp('2018-02-17 18:06:00'), et=Timestamp('2018-02-17 18:07:00'))
Pandas(Index=0, clusters=(237836700,), st=Timestamp('2018-02-18 13:57:00'), et=Timestamp('2018-02-18 13:58:00'))
Pandas(Index=0, clusters=(239299000,), st=Timestamp('2018-03-03 14:13:00'), et=Timestamp('2018-03-03 14:14:00'))
Pandas(Index=0, clusters=(241024000,), st=Timestamp('2018-03-06 18:26:00'), et=Timestamp('2018-03-06 18:27:00'))
Pandas(Index=0, clusters=(239634000,), st=Timestamp('2018-03-13 00:34:00'), et=Timestamp('2018-0

In [14]:
cluster=res_mcs
cluster = pd.DataFrame(res_mcs)
cluster['clusters'] = cluster['clusters'].str.get(0)
cluster.reset_index(drop=True, inplace=True)

In [65]:
#cluster.to_csv('./evolving_cluster_clusters_raw.csv', header=True, index=False ,sep=';')

## Visualize the results using ST-Visions

In [5]:
tmp=pd.read_csv('./data/ec_final.csv',header='infer',sep=';')
tmp=helper.getGeoDataFrame_v2(tmp, crs='epsg:4326')
tmp=tmp.rename(columns={'timestamp':'ts'})
tmp.loc[:, 'mmsi'] = tmp.mmsi.apply(str)

In [13]:
points = st_visualizer.st_visualizer()
points.set_data(tmp.copy())
points.create_canvas('Prototype Plot')
points.add_numerical_filter(filter_mode='<=', numeric_name='ts', step=60, callback_policy='value')
cmap = points.add_categorical_colormap('Category10', 'mmsi')
points.add_glyph(color=cmap, legend_group='mmsi')
points.add_map_tile(provider='CARTODBPOSITRON')
points.add_hover_tooltips([('mmsi', '@mmsi'), ('trip_id', '@trip_id'), ('timestamp', '@ts')])

points.show_figures(notebook=True, notebook_url='http://localhost:8889')

Loading BokehJS ...